Dataset Download and Visualization

In [2]:
from datasets import load_dataset

gtzan = load_dataset("marsyas/gtzan", "all")
gtzan

README.md:   0%|          | 0.00/4.42k [00:00<?, ?B/s]

gtzan.py:   0%|          | 0.00/3.35k [00:00<?, ?B/s]

The repository for marsyas/gtzan contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/marsyas/gtzan.
You can avoid this prompt in future by passing the argument `trust_remote_code=True`.

Do you wish to run the custom code? [y/N]  y


genres.tar.gz:   0%|          | 0.00/1.23G [00:00<?, ?B/s]

Generating train split: 0 examples [00:00, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['file', 'audio', 'genre'],
        num_rows: 999
    })
})

In [3]:
gtzan['train'][0]

{'file': '/root/.cache/huggingface/datasets/downloads/extracted/3b204381d6c029312e4f9c569c6b1130af3041dd36ca38ca53d4e20f585e39c6/genres/blues/blues.00000.wav',
 'audio': {'path': '/root/.cache/huggingface/datasets/downloads/extracted/3b204381d6c029312e4f9c569c6b1130af3041dd36ca38ca53d4e20f585e39c6/genres/blues/blues.00000.wav',
  'array': array([ 0.00732422,  0.01660156,  0.00762939, ..., -0.05560303,
         -0.06106567, -0.06417847]),
  'sampling_rate': 22050},
 'genre': 0}

In [4]:
gtzan = gtzan["train"].train_test_split(shuffle = True, seed = 42, test_size= 0.1)

In [5]:
gtzan

DatasetDict({
    train: Dataset({
        features: ['file', 'audio', 'genre'],
        num_rows: 899
    })
    test: Dataset({
        features: ['file', 'audio', 'genre'],
        num_rows: 100
    })
})

In [92]:
id2label_fn = gtzan['train'].features['genre'].int2str
id2label_fn(gtzan['train'][0]['genre'])

'pop'

In [7]:
!pip install gradio
# import gradio as gr

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.2/54.2 MB 32.5 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 323.1/323.1 kB 18.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.2/95.2 kB 6.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.6/11.6 MB 103.5 MB/s eta 0:00:0000:010:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.0/72.0 kB 5.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.5/62.5 kB 3.9 MB/s eta 0:00:00


In [93]:
import gradio as gr
def generate_audio():
    example = gtzan['train'].shuffle()[0]
    audio = example['audio']
    return (audio['sampling_rate'], audio['array']), id2label_fn(example['genre'])
with gr.Blocks() as demo:
    with gr.Column():
        for _ in range(3):
            audio, label = generate_audio()
            output = gr.Audio(audio, label= label)

demo.launch(debug=True)        

/usr/local/lib/python3.11/dist-packages/gradio/processing_utils.py:753: UserWarning: Trying to convert audio automatically from float64 to 16-bit int format.
  warnings.warn(warning.format(data.dtype))


* Running on local URL:  http://127.0.0.1:7860
It looks like you are running Gradio on a hosted a Jupyter notebook. For the Gradio app to work, sharing must be enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

* Running on public URL: https://a4524a67cdbd6729aa.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


Keyboard interruption in main thread... closing server.
Killing tunnel 127.0.0.1:7860 <> https://a4524a67cdbd6729aa.gradio.live


Feature Extractor

In [94]:
import transformers

In [95]:
from transformers import AutoFeatureExtractor
model_name = "ntu-spml/distilhubert"
feature_extractor = AutoFeatureExtractor.from_pretrained(model_name, do_normalize = True, return_attention_mask = True)


In [123]:
model_id = "ntu-spml/distilhubert"


In [96]:
feature_extractor

Wav2Vec2FeatureExtractor {
  "do_normalize": true,
  "feature_extractor_type": "Wav2Vec2FeatureExtractor",
  "feature_size": 1,
  "padding_side": "right",
  "padding_value": 0,
  "return_attention_mask": true,
  "sampling_rate": 16000
}

In [97]:
from datasets import Audio
gtzan = gtzan.cast_column("audio",Audio(sampling_rate = 16000))

In [98]:
gtzan['train'][0]['audio']

{'path': '/root/.cache/huggingface/datasets/downloads/extracted/3b204381d6c029312e4f9c569c6b1130af3041dd36ca38ca53d4e20f585e39c6/genres/pop/pop.00098.wav',
 'array': array([ 0.0873509 ,  0.20183384,  0.4790867 , ..., -0.18743178,
        -0.23294401, -0.13517427]),
 'sampling_rate': 16000}

Checking the mean and variance prior to applying feature extractor

In [100]:
import numpy as np

sample = gtzan["train"][0]["audio"]

print(f"Mean: {np.mean(sample['array']):.3}, Variance: {np.var(sample['array']):.3}")

Mean: 0.000185, Variance: 0.0493


In [101]:
inputs = feature_extractor(sample['array'], sampling_rate = sample['sampling_rate'] )

In [102]:
inputs

{'input_values': [array([ 0.3927024,  0.9084719,  2.1575537, ..., -0.8452508, -1.0502928,
       -0.6098198], dtype=float32)], 'attention_mask': [array([1, 1, 1, ..., 1, 1, 1], dtype=int32)]}

In [103]:
inputs
print(list[inputs.keys()])
print(
    f"Mean: {np.mean(inputs['input_values']):.3}, Variance: {np.var(inputs['input_values']):.3}"
)

list[dict_keys(['input_values', 'attention_mask'])]
Mean: -7.74e-09, Variance: 1.0


Now applying it to all the samples and truncating the audio of length >30s

In [104]:
MAX_LENGTH_AUDIO = 30.0

def process_audios(examples):
    example_arrays = [example['array'] for example in examples['audio']]
    inputs = feature_extractor(example_arrays,
                              sampling_rate = feature_extractor.sampling_rate,
                                truncation = True,
        max_length=int(feature_extractor.sampling_rate * MAX_LENGTH_AUDIO),
                               return_attention_mask = True
                              )
    return inputs

In [105]:
gtzan_encoded = gtzan.map(process_audios,
                         remove_columns = ['audio','file'],
                         batched = True,
                         batch_size = 100,
                         num_proc = 1,)

Map:   0%|          | 0/899 [00:00<?, ? examples/s]

Map:   0%|          | 0/100 [00:00<?, ? examples/s]

In [106]:
gtzan_encoded

DatasetDict({
    train: Dataset({
        features: ['genre', 'input_values', 'attention_mask'],
        num_rows: 899
    })
    test: Dataset({
        features: ['genre', 'input_values', 'attention_mask'],
        num_rows: 100
    })
})

In [108]:
gtzan_encoded = gtzan_encoded.rename_column("genre", "label")

ValueError: Original column name genre not in the dataset. Current columns in the dataset: ['label', 'input_values', 'attention_mask']

In [109]:
gtzan_encoded

DatasetDict({
    train: Dataset({
        features: ['label', 'input_values', 'attention_mask'],
        num_rows: 899
    })
    test: Dataset({
        features: ['label', 'input_values', 'attention_mask'],
        num_rows: 100
    })
})

In [23]:
gtzan_encoded['train'].features['Label'].names

['blues',
 'classical',
 'country',
 'disco',
 'hiphop',
 'jazz',
 'metal',
 'pop',
 'reggae',
 'rock']

In [110]:
id2label = {
    str(i): id2label_fn(i)
    for i in range(len(gtzan_encoded['train'].features['label'].names))
}
label2id = {v:k for k ,v in id2label.items() }
id2label["8"]

'reggae'

In [111]:
id2label["7"]

'pop'

Finetuning the model

In [112]:
from transformers import AutoModelForAudioClassification
num_labels = len(id2label)

model = AutoModelForAudioClassification.from_pretrained(
    model_name,
    num_labels = num_labels,
    label2id = label2id,
    id2label = id2label
)

Some weights of HubertForSequenceClassification were not initialized from the model checkpoint at ntu-spml/distilhubert and are newly initialized: ['classifier.bias', 'classifier.weight', 'projector.bias', 'projector.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [113]:
model

HubertForSequenceClassification(
  (hubert): HubertModel(
    (feature_extractor): HubertFeatureEncoder(
      (conv_layers): ModuleList(
        (0): HubertGroupNormConvLayer(
          (conv): Conv1d(1, 512, kernel_size=(10,), stride=(5,), bias=False)
          (activation): GELUActivation()
          (layer_norm): GroupNorm(512, 512, eps=1e-05, affine=True)
        )
        (1-4): 4 x HubertNoLayerNormConvLayer(
          (conv): Conv1d(512, 512, kernel_size=(3,), stride=(2,), bias=False)
          (activation): GELUActivation()
        )
        (5-6): 2 x HubertNoLayerNormConvLayer(
          (conv): Conv1d(512, 512, kernel_size=(2,), stride=(2,), bias=False)
          (activation): GELUActivation()
        )
      )
    )
    (feature_projection): HubertFeatureProjection(
      (projection): Linear(in_features=512, out_features=768, bias=True)
      (dropout): Dropout(p=0.0, inplace=False)
    )
    (encoder): HubertEncoder(
      (pos_conv_embed): HubertPositionalConvEmbedding(

In [52]:
# run in a notebook cell or terminal
!git config --global credential.helper store   # or:  cache --timeout=3600
!huggingface-cli login                         # or notebook_login() again



    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    A token is already saved on your machine. Run `huggingface-cli whoami` to get more information or `huggingface-cli logout` if you want to log out.
    Setting a new token will erase the existing one.
    To log in, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Enter your token (input will not be visible): Traceback (most recent call last):
  File "/usr/local/bin/huggingface-c

In [53]:
from huggingface_hub import notebook_login

notebook_login()

In [115]:
from transformers import TrainingArguments
model_name = model_name.split('/')[-1]

In [116]:
model_name

'distilhubert'

In [85]:
gradient_accumulation_steps = 1
batch_size = 4
num_train_epochs = 10

In [117]:
training_args = TrainingArguments(
    f"{model_name}-finetuned-gtzan",
    eval_strategy="epoch",
    save_strategy="epoch",
    learning_rate=5e-5,
    per_device_train_batch_size=batch_size,
    gradient_accumulation_steps=gradient_accumulation_steps,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=num_train_epochs,
    warmup_ratio=0.1,
    logging_steps=5,
    load_best_model_at_end=True,
    metric_for_best_model="accuracy",
    fp16=True,
    push_to_hub=True,
    report_to=["none"],
    label_names=["labels"]
)

In [87]:
# !pip install evaluate

In [118]:
import evaluate
import numpy as np

metric = evaluate.load("accuracy")

def compute_metrics(eval_pred):
    """Computes accuracy on a batch of predictions"""
    preds = np.argmax(eval_pred.predictions,axis = 1)
    return metric.compute(predictions = preds, references = eval_pred.label_ids)
    

In [119]:
metric

EvaluationModule(name: "accuracy", module_type: "metric", features: {'predictions': Value(dtype='int32', id=None), 'references': Value(dtype='int32', id=None)}, usage: """
Args:
    predictions (`list` of `int`): Predicted labels.
    references (`list` of `int`): Ground truth labels.
    normalize (`boolean`): If set to False, returns the number of correctly classified samples. Otherwise, returns the fraction of correctly classified samples. Defaults to True.
    sample_weight (`list` of `float`): Sample weights Defaults to None.

Returns:
    accuracy (`float` or `int`): Accuracy score. Minimum possible value is 0. Maximum possible value is 1.0, or the number of examples input, if `normalize` is set to `True`.. A higher score means higher accuracy.

Examples:

    Example 1-A simple example
        >>> accuracy_metric = evaluate.load("accuracy")
        >>> results = accuracy_metric.compute(references=[0, 1, 2, 0, 1, 2], predictions=[0, 1, 1, 2, 1, 0])
        >>> print(results)
    

In [120]:
from transformers import Trainer

trainer_instantiation  = Trainer(
    model,
    training_args,
    train_dataset = gtzan_encoded["train"],
    eval_dataset = gtzan_encoded["test"],
    tokenizer = feature_extractor,
    compute_metrics = compute_metrics,
)

/tmp/ipykernel_35/3937425063.py:3: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer_instantiation  = Trainer(


In [121]:
trainer_instantiation.train()

/usr/local/lib/python3.11/dist-packages/torch/nn/parallel/_functions.py:70: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(


Epoch,Training Loss,Validation Loss,Accuracy
1,1.955700,1.868564,0.400000
2,1.224500,1.290132,0.630000
3,1.106600,1.026510,0.700000
4,0.626400,0.842042,0.800000
5,0.579200,0.793611,0.770000
6,0.432200,0.680431,0.820000
7,0.342800,0.573944,0.840000
8,0.138500,0.603578,0.840000
9,0.241800,0.592568,0.840000
10,0.110500,0.598225,0.830000


/usr/local/lib/python3.11/dist-packages/torch/nn/parallel/_functions.py:70: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torch/nn/parallel/_functions.py:70: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torch/nn/parallel/_functions.py:70: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torch/nn/parallel/_functions.py:70: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torch/nn/parallel/_functions.py:70: UserWarning: Was asked t

TrainOutput(global_step=1130, training_loss=0.7790836344250536, metrics={'train_runtime': 5320.7677, 'train_samples_per_second': 1.69, 'train_steps_per_second': 0.212, 'total_flos': 6.133988274624e+17, 'train_loss': 0.7790836344250536, 'epoch': 10.0})

In [122]:
model_name

'distilhubert'

In [128]:
hf_username = "Metallic-HuH"


In [129]:
trainer.push_to_hub(
    repo_id = f"{hf_username}/{model_name}-finetuned-gtzan", 
    commit_message = "Add GTZAN-finetuned checkpoint",
)


model.safetensors:   0%|          | 0.00/94.8M [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/Metallic-HuH/distilhubert-finetuned-gtzan/commit/f6a3f2dd48f00bc8c84e58443edac509737c590c', commit_message='Add GTZAN-finetuned checkpoint', commit_description='', oid='f6a3f2dd48f00bc8c84e58443edac509737c590c', pr_url=None, repo_url=RepoUrl('https://huggingface.co/Metallic-HuH/distilhubert-finetuned-gtzan', endpoint='https://huggingface.co', repo_type='model', repo_id='Metallic-HuH/distilhubert-finetuned-gtzan'), pr_revision=None, pr_num=None)

In [ ]:
from transformers import Trainer

trainer = Trainer(
    model,
    training_args,
    train_dataset=gtzan_encoded["train"],
    eval_dataset=gtzan_encoded["test"],
    tokenizer=feature_extractor,
    compute_metrics=compute_metrics,
)

trainer.train()